<a href="https://colab.research.google.com/github/jwr0218/analysis_reviews_restaurant_with_Korean_English/blob/main/korean_TM_NB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install lda

In [ ]:

!pip install konlpy

In [ ]:
import nltk
import re
import collections
import itertools
import lda
import requests
import csv
import time
import math
import operator
import numpy as np
import pandas as pd
from collections import defaultdict
from pandas import read_table
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from konlpy  import tag 





In [ ]:
data = pd.read_csv('/content/drive/MyDrive/data_mango_merged.csv', encoding = 'utf-8')

In [ ]:
data

data['evaluation'] = data.apply(lambda x : 1 if x['evaluation'] == '맛있다' else x['evaluation'],axis = 1)


data['evaluation'] = data.apply(lambda x : 0 if x['evaluation'] == '별로' else x['evaluation'],axis = 1)


In [ ]:
total_review = data['review']
positive_review =  data[data['evaluation']==1]['review']


negative_review = data[data['evaluation']==0]['review']


tot_review = total_review.values.tolist()
pos_review = positive_review.values.tolist()
neg_review = negative_review.values.tolist()

In [ ]:
pos_word = ['같다', '맛있다', '나오다', '없다', '메뉴', '자다', '가다', '들다', '정도', '느낌', '방문', '생각', '싶다', '그렇다', '인데', '주문', '되어다', '좋아하다', '만들다', '느끼다', '조금', '식감', '오다', '재료', '살짝', '하나', '소스', '까지', '에는', '들어가다', '주다', '많다', '나다', '음식', '시키다', '정말', '처럼', '역시', '부드럽다', '느껴지다']

neg_word = ['가다', '같다', '맛있다', '없다', '들다', '인데', '까지', '정말', '아쉽다', '느낌', '주문', '알다', '가격', '이렇다', '괜찮다', '메뉴', '방문', '많다', '레스토랑', '자다', '이라고는', '아내', '방법', '별로', '그렇다', '주다', '껍질', '소스', '서비스', '예전', '이집', '에는', '식당', '예약', '게다가', '식감', '기억', '요리', '고기', '지금']

In [ ]:
data['evaluation'].unique()

In [ ]:
class SentenceTokenizer(object):
    
    def __init__(self):
        
        self.kkma = tag.Kkma()
        self.okt = tag.Okt()
        self.twitter = tag.Twitter()
        self.stopwords = ['중인' ,'만큼', '마찬가지', '꼬집었', "연합뉴스", "데일리", "동아일보", "중앙일보", "조선일보", "기자","아", "휴", "아이구", "아이쿠", "아이고", "어", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가",]
    
    
    def text2sentences(self, text):
        sentences = self.kkma.sentences(text)
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.twitter.nouns(str(sentence))
                                           if noun not in self.stopwords and len(noun) > 1]))
        return nouns
    
    
    def get_adj(self, sentences):
        adj = []
        count = dict()
        not_use_lst = ['좋다','보다','하다','먹다','있다','되다','이다','아니다','이다','하고','으로','에서','않다',
                       '에게','에서']
        for sentence in sentences:
            if sentence is not '':
                find = [a[0] for a in self.okt.pos(str(sentence),norm = True ,stem = True)
                                           if (a[1] in ['Adjective','Verb','Noun','Josa'] and a[0] not in not_use_lst) ]
                adj.append(" ".join(find))
                for i in find:
                    try:
                        count[i] = count[i] + 1
                    except KeyError:
                        count[i] = 1        
        return adj , count

In [ ]:
tokenizer = SentenceTokenizer()
okt = tag.Okt








#okt.pos(str(sentence),norm = True , stem = True)

total_tokenized , total_count = tokenizer.get_adj(total_review)

positive_tokenized ,positive_count = tokenizer.get_adj(positive_review)

negative_tokenized, negative_count = tokenizer.get_adj(negative_review)


In [ ]:
positive_tokenized

In [ ]:

service_good_feature = {'좋':0,'친절':0,'괜찮':0,'최고':0,'빠르':0,'짱':0,'훌륭':0,'추천':0,'감사':0,'구수':0,'최상':0,'대박':0,'훈훈':0,'특별':0,'개이득':0,'최고':0,'만족':0,'세련':0,'최고':0,'감동':0,
                        '스윗':0,'센스':0 ,'청결하다':0, '훌륭':0, '자부심':0
                        }

service_bad_feature = {'아쉽':0,'최악':0,'나쁘다':0,'느리':0,'빡치다':0,'비추':0,'별로':0,'그냥':0,'낙제':0,'엉망':0,'실망':0,'불친절':0,'문제':0,'컴플레인':0,'거지':0,'그닥':0,'그다지':0,'구려':0,'불편':0,'엉성':0,'헬':0,'개판':0,
                       '의외의':0,'불통':0,'':0,'불친절하다':0,'똑바로':0,'싸가지':0,'지나치다':0,'분하다':0,'민망하다':0,'잃다':0}
# 서비스 좋음 / 안좋음 
#맛
taste_good_feature = {'맛있다':0 ,'고급스럽다':0, '정말':0,'고소':0,'부드럽':0,'신선':0,'촉촉':0,'싱싱':0,'정갈':0,'존맛':0,'그리움':0,'만족':0,
                      '풍부':0,'바삭':0,'신선하다':0 , '담백':0, '즐기다':0,'추천' : 0, '건강하다':0, '싱싱':0,'달콤':0

                      }

taste_bad_feature = {'맛없다':0,'불호':0,'노맛':0,'심각':0,'없다':0,'평범':0,'찝찝하다':0
                     ,'싱겁':0,'느끼다하다':0,'짜다':0,'상해':0,'상하다':0,'느끼다':0,'짜다':0,'딱딱하다':0,'차갑다':0,'퍽퍽해':0, '퍽퍽':0,'모르다':0,'아쉽다':0,
                     '버리다':0,
                     '무난':0,'들쭉날쭉':0 , '싫어':0 }
#느낌 입감 
cost_good_feature = {'괜찮':0,'착하다':0,'저렴':0,'적당':0,'싸다':0,'좋다':0,'합리적':0,'훌륭':0,'최고':0,'만족':0,'마음':0,'든든':0,'알맞다':0,'무난':0,'괜춘':0,'최상':0,'낮':0
                     ,'많':0,'적당':0,'푸짐':0,'괜찮다':0,'넉넉':0,'충분':0    
                     }

cost_bad_feature ={'비싸':0,'있다':0,'나쁘':0,'사악':0,'비효율':0,'높다':0,'부담':0,'아쉽':0,'별로':0,'그닥':0,'그다지':0,'쎄':0,'높':0,'거품':0,'눈물':0,
                   '부족':0, '별로':0,'적다':0,'작다':0,'아쉽':0,'적다':0,'다소':0,'별로':0  
                   }
# 특징 
atmosphere_good_feature = {'괜찮':0,'조용':0,'깔끔':0,'적당':0,'깡패':0,'굉장':0,'아담':0,'완벽':0,'아기자기':0,'고급':0,'최고':0,'세련':0,'만족':0,'아늑':0,'훌륭':0 , '한적하다':0 ,'한적':0,
                           '좁다':0
                            ,'예쁘':0,'이쁘':0,'짱':0,'심쿵':0,'따뜻':0,'깨끗':0,'독특':0,'매력':0,'모던':0,'취향저격':0,'맘':0,'마음':0,'클래식':0,'아름':0,'인상적':0,'귀엽':0,'포근':0
                           ,'여유':0 

                           
                           
                           } 
                           
atmosphere_bad_feature = {'어수선하다':0,'부조화':0,'나쁘다':0,'바쁘다':0,'어수선하다':0,'이상하다':0,'촌스럽다':0,'별로':0,'부담스럽다':0,'시끄럽':0,'복잡':0 ,'어둡다':0,'불편':0,'과대':0,'애매':0,
                          '불쾌' : 0  
                          }
#분위기





In [ ]:
def keyword_count(review,dic):
  
  for i in dic.keys():


   
    
      
    if i in review:


      dic[i] +=1
  return dic 
        

In [ ]:
keyword_count(total_tokenized[0],service_good_feature)

In [ ]:
preprocessed_data_service = pd.DataFrame()
preprocessed_data_taste = pd.DataFrame()
preprocessed_data_cost = pd.DataFrame()
preprocessed_data_atmosphere = pd.DataFrame()

for token in total_tokenized:
  
  service_good_feature = {'좋':0,'친절':0,'괜찮':0,'최고':0,'빠르':0,'짱':0,'훌륭':0,'추천':0,'감사':0,'구수':0,'최상':0,'대박':0,'훈훈':0,'특별':0,'개이득':0,'최고':0,'만족':0,'세련':0,'최고':0,'감동':0,
                        '스윗':0,'센스':0 ,'청결하다':0, '훌륭':0, '자부심':0, 
                        }

  service_bad_feature = {'아쉽':0,'최악':0,'나쁘다':0,'느리':0,'빡치다':0,'비추':0,'별로':0,'그냥':0,'낙제':0,'엉망':0,'실망':0,'불친절':0,'문제':0,'컴플레인':0,'거지':0,'그닥':0,'그다지':0,'구려':0,'불편':0,'엉성':0,'헬':0,'개판':0,
                        '의외의':0,'불통':0,'':0,'불친절하다':0,'똑바로':0,'싸가지':0,'지나치다':0,'분하다':0,'민망하다':0,'잃다':0}
  # 서비스 좋음 / 안좋음 

  #맛
  taste_good_feature = {'맛있다':0 ,'고급스럽다':0, '정말':0,'고소':0,'부드럽':0,'신선':0,'촉촉':0,'싱싱':0,'정갈':0,'존맛':0,'그리움':0,'만족':0,
                        '풍부':0,'바삭':0,'신선하다':0 , '담백':0, '즐기다':0,'추천' : 0, '건강하다':0, '싱싱':0,'달콤':0

                        }

  taste_bad_feature = {'맛없다':0,'불호':0,'노맛':0,'심각':0,'없다':0,'평범':0,'찝찝하다':0
                      ,'싱겁':0,'느끼다하다':0,'짜다':0,'상해':0,'상하다':0,'느끼다':0,'짜다':0,'딱딱하다':0,'차갑다':0,'퍽퍽해':0, '퍽퍽':0,'모르다':0,'아쉽다':0,
                      '버리다':0,
                      '무난':0,'들쭉날쭉':0 , '싫어':0 }
  #느낌 입감 
  cost_good_feature = {'괜찮':0,'착하다':0,'저렴':0,'적당':0,'싸다':0,'좋다':0,'합리적':0,'훌륭':0,'최고':0,'만족':0,'마음':0,'든든':0,'알맞다':0,'무난':0,'괜춘':0,'최상':0,'낮':0
                      ,'많':0,'적당':0,'푸짐':0,'괜찮다':0,'넉넉':0,'충분':0                    
                      }

  cost_bad_feature ={'비싸':0,'있다':0,'나쁘':0,'사악':0,'비효율':0,'높다':0,'부담':0,'아쉽':0,'별로':0,'그닥':0,'그다지':0,'쎄':0,'높':0,'거품':0,'눈물':0,
                    '부족':0, '별로':0,'적다':0,'작다':0,'아쉽':0,'적다':0,'다소':0,'별로':0 
                    }
  #
  
  atmosphere_good_feature = {'괜찮':0,'조용':0,'깔끔':0,'적당':0,'깡패':0,'굉장':0,'아담':0,'완벽':0,'아기자기':0,'고급':0,'최고':0,'세련':0,'만족':0,'아늑':0,'훌륭':0 , '한적하다':0 ,'한적':0,
                            '좁다':0
                              ,'예쁘':0,'이쁘':0,'짱':0,'심쿵':0,'따뜻':0,'깨끗':0,'독특':0,'매력':0,'모던':0,'취향저격':0,'맘':0,'마음':0,'클래식':0,'아름':0,'인상적':0,'귀엽':0,'포근':0
                            ,'여유':0 

                            
                            
                            } 
                            
  atmosphere_bad_feature = {'어수선하다':0,'부조화':0,'나쁘다':0,'바쁘다':0,'어수선하다':0,'이상하다':0,'촌스럽다':0,'별로':0,'부담스럽다':0,'시끄럽':0,'복잡':0 ,'어둡다':0,'불편':0,'과대':0,'애매':0,
                            '불쾌' : 0  
                            }
  #분위기
  service_good_feature = keyword_count(token,service_good_feature)
  service_bad_feature = keyword_count(token,service_bad_feature)
  service_good_feature_df = pd.DataFrame(service_good_feature,index = [0])
  service_bad_feature_df = pd.DataFrame(service_bad_feature,index = [0])
  
  preprocessed_data_service_concated = pd.concat([service_good_feature_df,service_bad_feature_df],axis = 1)


  taste_good_feature = keyword_count(token,taste_good_feature)
  taste_bad_feature = keyword_count(token,taste_bad_feature)
  taste_good_feature_df = pd.DataFrame(taste_good_feature,index = [0])
  taste_bad_feature_df = pd.DataFrame(taste_bad_feature,index = [0])
  
  
  preprocessed_data_taste_concated = pd.concat([taste_good_feature_df,taste_bad_feature_df],axis = 1)

  
  
  cost_good_feature = keyword_count(token,cost_good_feature )
  cost_bad_feature = keyword_count(token,cost_bad_feature )
  cost_good_feature_df = pd.DataFrame(cost_good_feature,index = [0])
  cost_bad_feature_df = pd.DataFrame(cost_bad_feature,index = [0])
  
  preprocessed_data_cost_concated = pd.concat([cost_good_feature_df,cost_bad_feature_df],axis = 1)

  
  atmosphere_good_feature = keyword_count(token,atmosphere_good_feature)
  atmosphere_bad_feature = keyword_count(token,atmosphere_bad_feature)
  atmosphere_good_feature_df = pd.DataFrame(atmosphere_good_feature,index = [0])
  atmosphere_bad_feature_df = pd.DataFrame(atmosphere_bad_feature,index = [0])

  preprocessed_data_atmosphere_concated = pd.concat([atmosphere_good_feature_df,atmosphere_bad_feature_df],axis = 1)



  preprocessed_data_service = pd.concat([preprocessed_data_service,preprocessed_data_service_concated],ignore_index=True )
  preprocessed_data_taste = pd.concat([preprocessed_data_taste,preprocessed_data_taste_concated],ignore_index=True )
  preprocessed_data_cost = pd.concat([preprocessed_data_cost,preprocessed_data_cost_concated],ignore_index=True )
  preprocessed_data_atmosphere = pd.concat([preprocessed_data_atmosphere,preprocessed_data_atmosphere_concated],ignore_index=True )





In [ ]:
preprocessed_data_service['evaluation'] = data['evaluation'].copy()
preprocessed_data_service = preprocessed_data_service[preprocessed_data_service['evaluation']!='괜찮다']


preprocessed_data_taste['evaluation'] = data['evaluation'].copy()

preprocessed_data_taste = preprocessed_data_taste[preprocessed_data_taste['evaluation']!='괜찮다']

preprocessed_data_cost['evaluation'] = data['evaluation'].copy()

preprocessed_data_cost = preprocessed_data_cost[preprocessed_data_cost['evaluation']!='괜찮다']
preprocessed_data_atmosphere['evaluation'] = data['evaluation'].copy()

preprocessed_data_atmosphere = preprocessed_data_atmosphere[preprocessed_data_atmosphere['evaluation']!='괜찮다']

In [ ]:
preprocessed_data_atmosphere.iloc[:,:-1]

In [ ]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from imblearn.under_sampling import NearMiss
import pandas as pd

import math 

# 정확도 계산을 위한 함수

list = []
coeff = []

df = preprocessed_data_service
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=390, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA 
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score


In [ ]:

df = df.drop([''],axis = 'columns')



In [ ]:
df.shape[1]


In [ ]:
df.shape

In [ ]:
def elbow(X):
  sse = []
  silhouette = []
  for i in range(2,20):
    km  = KMeans(n_clusters = i , init = 'k-means++' , random_state =0 )
    km.fit(X)
    sse.append(km.inertia_)
    sil = silhouette_score(X,km.labels_   )
    silhouette.append(sil)
  plt.figure(1)
  plt.plot(range(2,20),sse , marker = 'o')
  plt.xlabel('number of cluster')
  plt.ylabel('SSE')
  plt.figure(2)
  plt.plot(range(2,20),silhouette , marker = 'x')
  plt.xlabel('number of cluster')
  plt.ylabel('silhouette')

  plt.show


  

In [ ]:

data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

local_km = KMeans(n_clusters = 12 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

service_df = df.copy()



In [ ]:
service_df

In [ ]:

coeff_service = pd.DataFrame(np.exp(b), columns= preprocessed_data_service.columns[:-1] )




In [ ]:
coeff_service

In [ ]:

list = []
coeff = []

df = preprocessed_data_taste
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=390, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:
data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

local_km = KMeans(n_clusters = 12)
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label

taste_df = df.copy()


In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

taste_df = df.copy()



In [ ]:

coeff_taste = pd.DataFrame(np.exp(b), columns= preprocessed_data_taste.columns[:-1] )




In [ ]:
coeff_taste

In [ ]:

list = []
coeff = []

df = preprocessed_data_cost
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=390, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:
data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

local_km = KMeans(n_clusters = 11 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label

cost_df = df.copy()

In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:
cost_df

In [ ]:

coeff_cost = pd.DataFrame(np.exp(b), columns= preprocessed_data_cost.columns[:-1] )




In [ ]:
coeff_cost

In [ ]:

list = []
coeff = []

df = preprocessed_data_atmosphere
for i in range(1000):
    data_ran = df[df['evaluation'].isin([1])]

    data_ran = data_ran.sample(n=390, random_state = 10)

    data_zero = df[df['evaluation'].isin([0])]

    df  = pd.concat([data_ran, data_zero])
    
   
    
    


    x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

    mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
    mod.fit(x_train, y_train)

    predicted = mod.predict(x_test)

    coeff.append( mod.feature_log_prob_ )


 
    list.append(accuracy_score(y_test, predicted))

import numpy as np

list_np = np.array(list)
a = np.mean(list_np)
b = np.mean(coeff,axis = 0)

print('score is',a)
print(b)

In [ ]:
data_for_kmeans = df.values
pca = PCA(n_components=7)

data_for_kmeans = pca.fit_transform(data_for_kmeans)
data_for_kmeans


elbow(data_for_kmeans)

local_km = KMeans(n_clusters = 12 )
local_km.fit(data_for_kmeans)

label = local_km.labels_
df['label_clustered'] = label

atmosphere_df = df.copy()

In [ ]:
pca = PCA(n_components=2)

data_for_kmeans = pca.fit_transform(data_for_kmeans)



plt.scatter(data_for_kmeans[:,0],data_for_kmeans[:,1],c = df['label_clustered'])
plt.show()

In [ ]:

coeff_atmosphere = pd.DataFrame(np.exp(b), columns= preprocessed_data_atmosphere.columns[:-1] )




In [ ]:
atmosphere_df

In [ ]:
coeff_atmosphere

In [ ]:
coeff_mean = np.mean(coeff,axis = 0)

In [ ]:
coeff_mean

In [ ]:
df = preprocessed_data_service


data_ran = df[df['evaluation'].isin([1])]

data_ran = data_ran.sample(n=390, random_state = 10)

data_zero = df[df['evaluation'].isin([0])]

df  = pd.concat([data_ran, data_zero])
    
   
    
    


x_train, x_test, y_train, y_test = model_selection.train_test_split(df.iloc[:,0:-1].values, df.iloc[:,-1].values.astype(int), test_size=0.3)

mod = BernoulliNB(alpha=1, class_prior=None, fit_prior=True)
mod.fit(x_train, y_train)

predicted = mod.predict(x_test)




In [ ]:
service_df.to_csv('service_df.csv',index = False)
taste_df.to_csv('taste_df.csv',index = False)
cost_df.to_csv('cost_df.csv',index = False)
atmosphere_df.to_csv('atmosphere_df.csv',index = False)



In [ ]:
lst =service_df['label_clustered'].unique().tolist()
lst = sorted(lst)

for i in lst: 

  find = service_df[service_df['label_clustered']==i].iloc[:,:-2]
  txt = find.sum().sort_values(ascending = False)[0:4]
  
  print("-------------------------")

  print("class  : {}".format(i))

  print(txt)

  print("-------------------------")




In [ ]:
lst =taste_df['label_clustered'].unique().tolist()
lst = sorted(lst)

for i in lst: 

  find = taste_df[taste_df['label_clustered']==i].iloc[:,:-2]
  txt = find.sum().sort_values(ascending = False)[0:4]
  
  print("-------------------------")

  print("class  : {}".format(i))

  print(txt)

  print("-------------------------")




In [ ]:
lst =cost_df['label_clustered'].unique().tolist()
lst = sorted(lst)

for i in lst: 

  find = cost_df[cost_df['label_clustered']==i].iloc[:,:-2]
  txt = find.sum().sort_values(ascending = False)[0:4]
  
  print("-------------------------")

  print("class  : {}".format(i))

  print(txt)

  print("-------------------------")


In [ ]:
lst =atmosphere_df['label_clustered'].unique().tolist()
lst = sorted(lst)

for i in lst: 

  find = atmosphere_df[atmosphere_df['label_clustered']==i].iloc[:,:-2]
  txt = find.sum().sort_values(ascending = False)[0:4]
  
  print("-------------------------")

  print("class  : {}".format(i))

  print(txt)

  print("-------------------------")


In [ ]:
find = service_df[service_df['label_clustered']==1].iloc[:,:-2]
find

In [ ]:
service_df.sum().sort_values(ascending = False)[2:6]

In [ ]:
ccc = '아 테스트 123123 keyword 상황 파악 어렵  '
re.findall('keyword +[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s', ccc)


In [ ]:

def turn_feature_df(dic):
  dic2 = {}
  for i in dic.keys():
    for j in dic[i].keys():
      dic2[i+"_"+j] = dic[i][j]

  return dic2

In [ ]:
def ngram2_count(review,dic):

'''
string = 'test test test'

if 'test in string:

print string



출처: https://metagenomics.tistory.com/entry/여러번-찾기 [메타지노믹스]
'''

  for i in dic.keys():

    a = re.findall(i +'+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s',review)
   
    b = re.findall('[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+[ㄱ-ㅎ|ㅏ-ㅣ|가-힣]+\s+'+i,review)
      
    if len(a)>0:
      print(a,"a 입니다.")
      splited_a = a[0].split()
      print(splited_a)
      try:
        dic[i][splited_a[1]] += 1

      except :

        pass
      try:
        dic[i][splited_a[2]] += 1

      except :
        pass
      try:
        dic[i][splited_a[3]] += 1

      except :
        pass
      
      try:
        dic[i][splited_a[4]] += 1

      except :
        pass
          
    elif len(b)>0:
        
        
      print(b,'B 입니다.')
      splited_b = b[0].split()
      print(splited_b)
      try:
        dic[i][splited_b[0]] += 1

      except :
        pass
      try:
        dic[i][splited_b[1]] += 1
      except :
        pass
      try:
        dic[i][splited_b[2]] += 1

      except :
        pass
      try:
        dic[i][splited_b[3]] += 1

      except :
        pass
        
          
  return dic

      
    
    

      


